# LinkedIn Job Hunter Agent v2

This notebook implements an agentic workflow using:
- **deepagents** from LangChain for agent orchestration with planning capabilities
- **Playwright MCP server** for browser automation
- **FilesystemBackend** to offload context to disk and manage memory
- **Custom middleware** to handle console noise and truncate redundant context

## Key Features:
1. Context management via FilesystemBackend - stores tool outputs on disk
2. Console noise reduction using `--console-level error`
3. Context window optimization using ClearToolUsesEdit middleware
4. Custom response truncation middleware

In [1]:
# Setup path for custom utilities
import sys
LLM_UTILS_PATH = "/Users/anuragmishra/Documents/AI Workspace/"
sys.path.insert(0, LLM_UTILS_PATH)

In [2]:
# Core imports
import os
import re
import asyncio
from typing import Any
from datetime import datetime

# LangChain and deepagents imports
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.agents.middleware import (
    AgentMiddleware,
    AgentState,
    ContextEditingMiddleware, 
    ClearToolUsesEdit,
)
from langgraph.runtime import Runtime

# MCP imports
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools

# LLM configuration
from llm_utils.models_config import openai_llm

## Configuration

Configure Playwright MCP server with optimized settings:
- `--console-level error` - Only capture error-level console messages to reduce noise
- `--headless` - Optional: run in headless mode (remove for visible browser)

In [4]:
# Playwright MCP server configuration
# Using --console-level error to filter out non-error console messages
# This significantly reduces context noise from website JavaScript logs
server_params = StdioServerParameters(
    command="npx",
    args=[
        "@playwright/mcp@latest",
        "--console-level", "error",  # Only capture errors, not info/warning/debug
        # "--headless",  # Uncomment for headless mode
    ],
)

# Directory to store agent filesystem data
AGENT_WORKSPACE_DIR = "/Users/anuragmishra/Documents/AI Workspace/web_agent/agent_workspace"
os.makedirs(AGENT_WORKSPACE_DIR, exist_ok=True)

## Custom Middleware for Context Management

These middleware components help manage the LLM context window:
1. **ResponseTruncationMiddleware** - Truncates long tool responses to prevent context overflow
2. **SnapshotCleanerMiddleware** - Cleans repeated accessibility snapshots from Playwright

In [5]:
class ResponseTruncationMiddleware(AgentMiddleware):
    """
    Middleware to truncate long tool responses.
    
    Websites often produce large accessibility snapshots or console output.
    This middleware truncates content exceeding max_chars to prevent
    context window overflow.
    """
    
    def __init__(self, max_chars: int = 3000, summary_chars: int = 500):
        self.max_chars = max_chars
        self.summary_chars = summary_chars
    
    def _truncate_content(self, content: str) -> str:
        """Truncate content and add marker."""
        if len(content) <= self.max_chars:
            return content
        
        # Keep beginning and end for context
        half = self.max_chars // 2
        return (
            content[:half] + 
            f"\n\n... [TRUNCATED {len(content) - self.max_chars} characters] ...\n\n" +
            content[-half:]
        )
    
    def before_model(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
        """Process messages before sending to model."""
        for msg in state['messages']:
            try:
                content = getattr(msg, 'content', None) or msg.get('content')
            except (AttributeError, TypeError):
                continue
            
            if isinstance(content, str) and len(content) > self.max_chars:
                truncated = self._truncate_content(content)
                try:
                    msg.content = truncated
                except AttributeError:
                    msg['content'] = truncated
            
            elif isinstance(content, list):
                for i, elem in enumerate(content):
                    if isinstance(elem, str) and len(elem) > self.max_chars:
                        content[i] = self._truncate_content(elem)
                    elif isinstance(elem, dict) and isinstance(elem.get('text'), str):
                        if len(elem['text']) > self.max_chars:
                            elem['text'] = self._truncate_content(elem['text'])
        
        return None
    
    def after_model(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
        return None

In [6]:
class SnapshotCleanerMiddleware(AgentMiddleware):
    """
    Middleware to clean redundant accessibility snapshots.
    
    Playwright MCP returns detailed accessibility trees that can be repetitive.
    This middleware identifies and condenses repeated snapshot patterns.
    """
    
    def __init__(self):
        self.seen_snapshots = set()
        self.snapshot_count = 0
    
    def _get_snapshot_hash(self, content: str) -> str:
        """Create a hash of snapshot structure (ignoring dynamic values)."""
        # Remove dynamic values like timestamps, IDs
        cleaned = re.sub(r'ref="[^"]+"', 'ref="..."', content)
        cleaned = re.sub(r'\d{10,}', '...', cleaned)
        return hash(cleaned[:1000])  # Hash first 1000 chars for structure
    
    def _is_snapshot(self, content: str) -> bool:
        """Check if content looks like a Playwright accessibility snapshot."""
        snapshot_indicators = [
            '- button',
            '- link',
            '- textbox',
            'ref="',
            '- heading',
        ]
        return any(indicator in content for indicator in snapshot_indicators)
    
    def before_model(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
        """Clean repeated snapshots from messages."""
        for msg in state['messages']:
            try:
                content = getattr(msg, 'content', None) or msg.get('content')
            except (AttributeError, TypeError):
                continue
            
            if isinstance(content, str) and self._is_snapshot(content):
                snapshot_hash = self._get_snapshot_hash(content)
                
                if snapshot_hash in self.seen_snapshots:
                    # Replace with condensed marker
                    condensed = "[Similar page snapshot - see previous snapshot for structure]"
                    try:
                        msg.content = condensed
                    except AttributeError:
                        msg['content'] = condensed
                else:
                    self.seen_snapshots.add(snapshot_hash)
                    self.snapshot_count += 1
                    # Keep only last 3 unique snapshots
                    if len(self.seen_snapshots) > 3:
                        self.seen_snapshots.pop()
        
        return None
    
    def after_model(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
        return None

In [7]:
class LoggingMiddleware(AgentMiddleware):
    """
    Middleware for logging and debugging agent execution.
    
    Logs message flow with truncated content preview.
    """
    
    def __init__(self, preview_length: int = 200):
        self.preview_length = preview_length
        self.call_count = 0
    
    def before_model(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
        self.call_count += 1
        print(f"\n{'='*60}")
        print(f"Model Call #{self.call_count} | Messages: {len(state['messages'])}")
        print(f"{'='*60}")
        
        # Show last message preview
        if state['messages']:
            last_msg = state['messages'][-1]
            try:
                content = getattr(last_msg, 'content', None) or last_msg.get('content', '')
                if isinstance(content, str):
                    preview = content[:self.preview_length]
                    if len(content) > self.preview_length:
                        preview += f"... [{len(content)} total chars]"
                    print(f"Last message preview: {preview}")
            except Exception:
                pass
        
        return None
    
    def after_model(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
        last_msg = state['messages'][-1]
        
        # Log tool calls if present
        tool_calls = getattr(last_msg, 'tool_calls', None)
        if tool_calls:
            tool_names = [tc.get('name', 'unknown') for tc in tool_calls]
            print(f"Tools called: {tool_names}")
        else:
            content = getattr(last_msg, 'content', '')[:100]
            print(f"Response: {content}...")
        
        return None

## Agent System Prompt

Optimized system prompt for LinkedIn job hunting with context management instructions.

In [8]:
AGENT_SYSTEM_PROMPT = """
You are an expert web automation agent specialized in job hunting on LinkedIn.

## Your Capabilities:
- Navigate websites using browser tools
- Fill forms and interact with UI elements
- Extract and organize job posting information
- Use the filesystem to store and retrieve data

## Context Management (CRITICAL):
To avoid context overflow, you MUST:
1. Use `write_file` to save large outputs (job listings, search results) to files
2. Use `read_file` to retrieve saved data when needed
3. Keep summaries in memory, details on filesystem
4. After capturing page snapshots, extract ONLY relevant elements

## File Organization:
- `/jobs/` - Store job listings as JSON files
- `/logs/` - Store session logs and progress
- `/data/` - Store extracted data and summaries

## Workflow Steps:
1. **Plan**: Use `write_todos` to create a step-by-step plan
2. **Navigate**: Go to linkedin.com and verify login status
3. **Search**: Search for specified job titles
4. **Extract**: For each job listing:
   - Extract: title, company, location, salary (if shown), posted date
   - Save to filesystem: `write_file("/jobs/{company}_{title}.json", job_data)`
5. **Summarize**: Create summary in `/data/summary.md`

## Important Guidelines:
- Always confirm login before searching
- If page content is too long, save it to a file first
- Handle pagination to get more results
- Update todos as you progress
- If you encounter errors, log them and adapt your plan

## Tool Usage Tips:
- `browser_snapshot` - Returns page accessibility tree (can be large)
- `browser_click` - Click elements by ref from snapshot
- `browser_type` - Type into input fields
- `browser_navigate` - Go to a URL
- `write_file` - ALWAYS use this for large data
- `read_file` - Retrieve saved data
"""

## Main Agent Execution

This creates and runs the deep agent with all middleware configured.

In [10]:
# Define the job search task
job_search_task = """
Go to linkedin.com and verify you are logged in.

Once confirmed logged in:
1. Search for 'Generative AI Architect' jobs
2. Filter for jobs posted in the last week
3. Extract the first 5 job postings with:
   - Job title
   - Company name
   - Location
   - Posted date
   - Brief description (first 100 words)
4. Save each job to the filesystem as JSON
5. Create a summary markdown file with all findings

Remember: Save large outputs to files to manage context!
"""

In [11]:
async def run_job_hunter_agent():
    """
    Main function to run the LinkedIn job hunter agent.
    
    Uses:
    - FilesystemBackend: Persists agent data to disk
    - Multiple middleware layers for context optimization
    - Playwright MCP with reduced console noise
    """
    
    print(f"Starting LinkedIn Job Hunter Agent")
    print(f"Workspace: {AGENT_WORKSPACE_DIR}")
    print(f"Timestamp: {datetime.now().isoformat()}")
    print("="*60)
    
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize MCP connection
            await session.initialize()
            
            # Load Playwright browser tools
            playwright_tools = await load_mcp_tools(session)
            print(f"Loaded {len(playwright_tools)} browser tools")
            print(f"Tools: {[t.name for t in playwright_tools[:5]]}...")
            
            # Create the deep agent with optimized configuration
            agent = create_deep_agent(
                model=openai_llm,
                tools=playwright_tools,
                system_prompt=AGENT_SYSTEM_PROMPT,
                
                # FilesystemBackend: Stores files to disk instead of memory
                # This allows the agent to offload large data and retrieve later
                backend=FilesystemBackend(
                    root_dir=AGENT_WORKSPACE_DIR,
                    virtual_mode=False  # Actually write to disk
                ),
                
                # Middleware stack for context management
                middleware=[
                    # 1. Logging for debugging
                    LoggingMiddleware(preview_length=150),
                    
                    # 2. Truncate overly long responses
                    ResponseTruncationMiddleware(
                        max_chars=3000,  # Max chars per message
                        summary_chars=500
                    ),
                    
                    # 3. Clean repeated snapshots
                    SnapshotCleanerMiddleware(),
                    
                    # 4. Clear old tool uses to manage context window
                    ContextEditingMiddleware(
                        edits=[
                            ClearToolUsesEdit(
                                trigger=4000,  # Trigger when context exceeds this
                                keep=5,  # Keep last 5 tool interactions
                                clear_tool_inputs=True,  # Also clear inputs
                                exclude_tools=['write_file', 'read_file'],  # Keep filesystem
                                placeholder="[Previous tool output cleared for context management]"
                            ),
                        ],
                    ),
                ],
            )
            
            print("\nAgent created. Starting execution...\n")
            
            # Run the agent with streaming
            final_response = None
            async for chunk in agent.astream(
                {"messages": [{"role": "user", "content": job_search_task}]},
                stream_mode="values"
            ):
                latest_message = chunk["messages"][-1]
                
                # Display progress
                if hasattr(latest_message, 'content') and latest_message.content:
                    if not hasattr(latest_message, 'tool_calls') or not latest_message.tool_calls:
                        final_response = latest_message.content
                        print(f"\n--- Agent Response ---")
                        print(latest_message.content[:500])
                        if len(latest_message.content) > 500:
                            print("...")
                
                elif hasattr(latest_message, 'tool_calls') and latest_message.tool_calls:
                    for tc in latest_message.tool_calls:
                        print(f"Executing: {tc.get('name', 'unknown')}")
            
            print("\n" + "="*60)
            print("Agent execution completed.")
            print(f"Check workspace for saved files: {AGENT_WORKSPACE_DIR}")
            
            return final_response

In [12]:
# Run the agent
result = await run_job_hunter_agent()

Starting LinkedIn Job Hunter Agent
Workspace: /Users/anuragmishra/Documents/AI Workspace/web_agent/agent_workspace
Timestamp: 2025-12-29T11:49:56.424990
Loaded 22 browser tools
Tools: ['browser_close', 'browser_resize', 'browser_console_messages', 'browser_handle_dialog', 'browser_evaluate']...

Agent created. Starting execution...


--- Agent Response ---

Go to linkedin.com and verify you are logged in.

Once confirmed logged in:
1. Search for 'Generative AI Architect' jobs
2. Filter for jobs posted in the last week
3. Extract the first 5 job postings with:
   - Job title
   - Company name
   - Location
   - Posted date
   - Brief description (first 100 words)
4. Save each job to the filesystem as JSON
5. Create a summary markdown file with all findings

Remember: Save large outputs to files to manage context!


--- Agent Response ---

Go to linkedin.com and verify you are logged in.

Once confirmed logged in:
1. Search for 'Generative AI Architect' jobs
2. Filter for jobs posted in 

  + Exception Group Traceback (most recent call last):
  |   File "/Users/anuragmishra/Documents/AI Workspace/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3699, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/var/folders/v2/2mlxtbzx4m5_nyk5t6yzxksw0000gn/T/ipykernel_6897/3874615398.py", line 2, in <module>
  |     result = await run_job_hunter_agent()
  |              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/var/folders/v2/2mlxtbzx4m5_nyk5t6yzxksw0000gn/T/ipykernel_6897/907291082.py", line 16, in run_job_hunter_agent
  |     async with stdio_client(server_params) as (read, write):
  |                ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/opt/homebrew/Cellar/python@3.12/3.12.12/Frameworks/Python.framework/Versions/3.12/lib/python3.12/contextlib.py", line 231, in __aexit__
  |     await self.gen.athrow(value)
  |   File "/Users/anuragmishra/Documents/AI Workspace/.venv/lib/python3.12/site-packages/mcp/client/stdio/

## Check Agent Outputs

After execution, check the files created by the agent.

In [ ]:
# List files created by the agent
import os

def list_workspace_files(directory):
    """Recursively list all files in the workspace."""
    for root, dirs, files in os.walk(directory):
        level = root.replace(directory, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 2 * (level + 1)
        for file in files:
            filepath = os.path.join(root, file)
            size = os.path.getsize(filepath)
            print(f'{subindent}{file} ({size} bytes)')

print(f"\nFiles in agent workspace:")
print("="*40)
list_workspace_files(AGENT_WORKSPACE_DIR)

In [ ]:
# Read summary if created
summary_path = os.path.join(AGENT_WORKSPACE_DIR, "data", "summary.md")
if os.path.exists(summary_path):
    with open(summary_path, 'r') as f:
        print("Job Search Summary:")
        print("="*40)
        print(f.read())
else:
    print("No summary file found yet.")

## Alternative: Using MultiServerMCPClient

For scenarios where you need stateful sessions or multiple MCP servers.

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient

async def run_with_multi_client():
    """
    Alternative approach using MultiServerMCPClient.
    
    Benefits:
    - Supports multiple MCP servers
    - Cleaner session management
    - Built-in stateful session support
    """
    
    # Configure MCP servers
    client = MultiServerMCPClient({
        "playwright": {
            "transport": "stdio",
            "command": "npx",
            "args": [
                "@playwright/mcp@latest",
                "--console-level", "error",
            ],
        }
    })
    
    # Get tools from all configured servers
    tools = await client.get_tools()
    print(f"Loaded {len(tools)} tools from MCP servers")
    
    # Create agent with tools
    agent = create_deep_agent(
        model=openai_llm,
        tools=tools,
        system_prompt=AGENT_SYSTEM_PROMPT,
        backend=FilesystemBackend(root_dir=AGENT_WORKSPACE_DIR),
        middleware=[
            ResponseTruncationMiddleware(max_chars=3000),
            ContextEditingMiddleware(
                edits=[
                    ClearToolUsesEdit(trigger=4000, keep=5),
                ],
            ),
        ],
    )
    
    # Execute
    async for chunk in agent.astream(
        {"messages": [{"role": "user", "content": job_search_task}]},
        stream_mode="values"
    ):
        latest_message = chunk["messages"][-1]
        if hasattr(latest_message, 'content') and latest_message.content:
            print(f"Agent: {latest_message.content[:200]}...")

# Uncomment to run with MultiServerMCPClient
# await run_with_multi_client()

## Documentation References

Based on official LangChain documentation:

### Deep Agents
- [Overview](https://docs.langchain.com/oss/python/deepagents/overview)
- [Middleware](https://docs.langchain.com/oss/python/deepagents/middleware)
- [Backends](https://docs.langchain.com/oss/python/deepagents/backends)
- [Customization](https://docs.langchain.com/oss/python/deepagents/customization)

### MCP Integration
- [LangChain MCP](https://docs.langchain.com/oss/python/langchain/mcp)

### Playwright MCP Server
- [GitHub](https://github.com/microsoft/playwright-mcp)
- [npm](https://www.npmjs.com/package/@playwright/mcp)